<h1>Workshop n°2 di GGPL di Emanuele Chiossi, mat. 439468</h1>
<h3>Parametric spatial building frame in reinforced concrete</h3>


<h2>Descrizione</h2>

L'obiettivo di questo workshop è quello di reallizzare una struttura in cemento armato, partendo dal risultato ottenuto dal workshop_01.

Come prima cosa, occorre prendere una immagine dal web che mi dia un'idea su come realizzare la struttura, ho scelto la seguente:

![alt text](model.jpg)


L'input della funzione che costruisce la struttura è rappresentato da un file .csv, in cui nelle linee dispari ci sono i vettori di traslazione tridimensionali, mentre nelle linee pari ci sono i parametri di riferimento del telaio piano.

Tramite le funzioni STRUCT, PROD e QUOTE si costruiscono i pilastri e le travi, il codice è semplice e la procedura adottata nel particolare per la costruzione dei singoli elementi è facilmente intuibile, ho seguito rigidamente le specifiche.

Le variabili utilizzate sono le seguenti:
<ul>
<li>beamSection:(bx,bz) sezione della trave</li>
<li>pillarSection:(px,py) sezione del pilastro</li>
<li>distanceBetweenPillar: distanza tra i pilastri</li>
<li>interstoryHeight: distanza tra i piani della struttura</li>
<li>distance: distanza dal pilastro X all'origine degli assi</li>
<li>x/y/zPillar: coordinate x,y,z pilastri</li>
<li>x/y/zBeam: coordinate x,y,z travi</li>
<li>pil,pillar: variabile che identifica i pilastri</li>
<li>bea,beam: variabile che identifica le travi</li>
<li>frame: variabile che identifica la struttura</li>
<li>transactions: vettore delle traslazioni</li>
<li>orizontalBeam: variabile che identifica le travi orizzontali</li>
<li>structure: variabile che identifica la struttura senza travi orizzontali</li>
<li>structure: variabile che identifica la struttura con travi orizzontali aggiunte</li>
</ul>

Altre decisioni implementative sono presenti sotto forma di commento nel codice.

Codice:

In [1]:
from pyplasm import *
import csv

"""funzione che realizza un telaio piano prendendo in input la sezione della trave, la sezione del pilastro, la distanza tra i pilastri e la distanza tra i piani"""

def createFrame(beamSection,pillarSection,distanceBetweenPillar,interstoryHeight):
    px = pillarSection[0]
    py = pillarSection[1]
    bx = beamSection[0]
    bz = beamSection[1]
    xPillar = []
    yPillar = []
    zPillar = []
    xBeam = []
    yBeam = []
    zBeam = [] 
    distance = 0  
    """ciclo che costruisce i pilastri alla giusta distanza"""
    for i in distanceBetweenPillar:
        xPillar = [px]
        yPillar = yPillar + [py,-i]
        distance = distance + i + py   
    yPillar = yPillar + [py]
    """ciclo che costruisce le travi alla giusta distanza"""
    for i in interstoryHeight:
        zPillar = zPillar + [i,-bz] 
        xBeam = [bx]
        yBeam = [distance+py]
        zBeam = zBeam + [-i,bz]
    pil = PROD([QUOTE(xPillar),QUOTE(yPillar)])
    pillar = PROD([pil,QUOTE(zPillar)])
    bea = PROD([QUOTE(xBeam),QUOTE(yBeam)])
    beam = PROD([bea,QUOTE(zBeam)])
    frame = STRUCT([pillar,beam])
    return frame

def createOrizzontalBeam(bx,bz,px,py,distanceBetweenPillar,interstoryHeight):
    xBeam = [bx]
    yBeam = []
    zBeam = []
    """ciclo che costruisce le travi ortogonali a quelle costruite precedentemente"""
    for i in interstoryHeight:
        zBeam = zBeam + [-i,bz]
    for i in distanceBetweenPillar:
        yBeam = yBeam + [py,-i]
    yBeam = yBeam + [py]
    bea = PROD([QUOTE(xBeam),QUOTE(yBeam)])
    beam = PROD([bea,QUOTE(zBeam)])
    frame = STRUCT([beam])
    return frame

def ggpl_bone_structure(fileName):
    """L'input della funzione è rappresentato da un file .csv, in cui nelle linee dispari ci sono i vettori di traslazione tridimensionali,
       e nelle linee pari ci sono i parametri di riferimento del telaio piano"""
    x = []
    y = []
    z = []
    frames = []
    transactions = []
    orizzontalBeam = []
    structure = []
    structure2 = []
    Vx=0
    """apertura del file e elaborazione dei dati"""
    with open(fileName,'rU') as csvfile:
        reader = csv.DictReader(csvfile)
        cont = 2 #contatore righe
        for row in reader:
            if(cont%2 != 0): #riga dispari
                beamSection = row['x'].replace("(","").replace(")","").split(";")
                pillarSection = row['y'].replace("(","").replace(")","").split(";")
                beamSectionX = eval(beamSection[0])
                beamSectionZ = eval(beamSection[1])
                pillarSectionX = eval(pillarSection[0])
                pillarSectionY = eval(pillarSection[1])
                distanceBetweenPillar = []
                for elem in row['z'].replace("[","").replace("]","").split(";"):
                    distanceBetweenPillar.append(eval(elem))
                interstoryHeight = []
                for elem in row['k'].replace("[","").replace("]","").split(";"):
                    interstoryHeight.append(eval(elem))
                orizzontalBeam.append(createOrizzontalBeam(Vx,beamSectionZ,pillarSectionX,pillarSectionY,distanceBetweenPillar,interstoryHeight))                                                             
                frames.append(createFrame((beamSectionX,beamSectionZ),(pillarSectionX,pillarSectionY),distanceBetweenPillar,interstoryHeight))
            if(cont%2 == 0): #riga pari
                Vx = x = eval(row['x'])
                y = eval(row['y'])
                z = eval(row['z'])
                transactions.append(T([1,2,3])([x,y,z]))
            cont = cont+1
    i = 0
    """costruzione della espressione che verrà poi mandata in input alla funzione STRUCT"""
    for elem in frames:
        structure.append(transactions[i])
        structure.append(frames[i])
        if(i+1 < len(frames)):
            structure2.append(transactions[i])
            structure2.append(orizzontalBeam[i+1])
        i = i+1
    view1 = STRUCT(structure)
    view2 = STRUCT(structure2)
    VIEW(STRUCT([view1,view2]))

"""lancio del programma"""
ggpl_bone_structure("frame_data_439468.csv")

Evaluating fenvs.py..
...fenvs.py imported in 0.011629 seconds


<h2>Screenshot dell'oggetto HPC ottenuto: </h2>


![alt text](structure.png)